<img src="./lecture_image/00_title.png">

<table border=1 width=100%>
    <tr><td style="border: 1px solid black; width:600px; height:30px; text-align: center;"><font size=4 color=blue><b>[21차시] 학습목표</b></font></td></tr>       
    <tr><td style="border: 1px solid black; text-align: left;"><font size=3>
        
○ Mediapipe 라이브러리를 활용할 수 있다 <br><br>
○ Mediapipe 라이브러리를 이용하여 얼굴 특성을 추출할 수 있다 <br><br>
○ Mediapipe 라이브러리를 이용하여 동작 특성을 추출할 수 있다 </font></td></tr>   
</table>

# Mediapipe 라이브러리를 이용한 얼굴 3D 데이터 추출

- 라이브 및 스트리밍 미디어를 위한 플랫폼 간 사용자 지정 가능한 ML 솔루션을 제공
- mediapipe 라이브러리 : https://google.github.io/mediapipe/getting_started/python
- 기능
<table width=800>
    <tr>
        <td><center><b>얼굴 검출</b></center></td>
        <td><center><b>얼굴 Mesh</b></center></td>
        <td><center><b>홍채 검출</b></center></td>
        <td><center><b>손동작 검출</b></center></td>
        <td><center><b>동작 검출</b></center></td>
        <td><center><b>전체동작</b></center></td>
    </tr>     
    <tr>
        <td><img src="./lecture_image/21_mediapipe_face_detection.gif"></td>
        <td><img src="./lecture_image/21_mediapipe_face_mesh.gif"></td>
        <td><img src="./lecture_image/21_mediapipe_iris_tracking.gif "></td>
        <td><img src="./lecture_image/21_mediapipe_hand_tracking.gif"></td>
        <td><img src="./lecture_image/21_mediapipe_pose_tracking.gif"></td>
        <td><img src="./lecture_image/21_mediapipe_holistic_tracking.gif"></td>
    </tr>    
    <tr>
        <td><center><b>헤어 분리</b></center></td>
        <td><center><b>객체 검출</b></center></td>
        <td><center><b>박스 추적</b></center></td>
        <td><center><b>모션 추적</b></center></td>
        <td><center><b>3D객체 검출</b></center></td>
        <td><center><b>템플릿 매칭</b></center></td>
    </tr> 
    <tr>
        <td><img src="./lecture_image/21_mediapipe_hair_segmentation.gif"></td>        
        <td><img src="./lecture_image/21_mediapipe_object_detection.gif"></td>
        <td><img src="./lecture_image/21_mediapipe_box_tracking.gif"></td>
        <td><img src="./lecture_image/21_mediapipe_instant_motion_tracking.gif"></td>
        <td><img src="./lecture_image/21_mediapipe_objectron_chair.gif"></td>
        <td><img src="./lecture_image/21_mediapipe_template_matching.gif"></td>
    </tr> 
</table>    

In [ ]:
# 프롬프트 창에서 설치
!pip install mediapipe

In [ ]:
import cv2
import mediapipe as mp

# No module named 'mediapipe'가 뜨는 경우
#   - 설치 폴더 (아래 설치내용 확인) 확인해서 .\anaconda3\envs\opencv\lib\site-packages로 이동

# No module named 'mediapipe.python._framework_bindings가 뜨는 경우
#   - 

## 얼굴 메시 추출

In [ ]:
#실습

import cv2
import mediapipe as mp

# 얼굴 메시를 추출하는 라이브러리 가져오기
# 결과를 시각화하는 라이브러리
mp_drawing = mp.solutions.drawing_utils
# 얼굴 메시를 추출하는 라이브러리 가져오기
mp_face_mesh = mp.solutions.face_mesh
# 얼굴, 홍채 등 찾는 객체에 맞는 라이브러리를 가져오면 됨

#시각화 설정(선 두께, 원의 반지름)
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# 영상 불러오기
cap=cv2.VideoCapture("./image/face3.mp4")

# 얼굴 메시 추출(최소 검출 정확도, 최소 추적 정확도)
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5,
                          min_tracking_confidence=0.5) as face_mesh :
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret :
            break
            
        # 이미지 쓰기 불가 설정 (속도때문에...)
        frame.flags.writeable = False
        
        # 메시 검출
        results = face_mesh.process(frame)
        
        frame.flags.writeable = True
        
        # 랜드마크를 서로 연결 (매시)
        # 랜드마크 점이 검출되었다면
        if results.multi_face_landmarks :
            print(results.multi_face_landmarks)
            # 검출 랜드마크를 하나씩 가져온다.
            for face_landmark in results.multi_face_landmarks :
                # 랜드마크들끼리 선으로 연결
                mp_drawing.draw_landmarks(
                    image=frame,
                landmark_list=face_landmark,
                # 얼굴 전체
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                # 그리기 설정
                landmark_drawing_spec=drawing_spec,
                connection_drawing_spec=drawing_spec
            )
        cv2.imshow("face mesh", frame)
        
        key = cv2.waitKey(33)
        
        if key == 49 :
            break
            
cap.release()
cv2.destroyAllWindows()

### 얼굴 매시 + 손 스켈레톤 검출 

In [2]:
#실습

import cv2
import mediapipe as mp

# 얼굴 메시를 추출하는 라이브러리 가져오기
# 결과를 시각화하는 라이브러리
mp_drawing = mp.solutions.drawing_utils
# 얼굴 메시를 추출하는 라이브러리 가져오기
mp_face_mesh = mp.solutions.face_mesh
# 손 스켈레톤 추출 라이브러리 가져오기
mp_hands = mp.solutions.hands
mp_drawing_styles = mp.solutions.drawing_styles
# 얼굴, 홍채 등 찾는 객체에 맞는 라이브러리를 가져오면 됨

#시각화 설정(선 두께, 원의 반지름)
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# 영상 불러오기
cap=cv2.VideoCapture(0)

# 얼굴 메시 추출(최소 검출 정확도, 최소 추적 정확도)
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5,
                          min_tracking_confidence=0.5) as face_mesh :
    with mp_hands.Hands(model_complexity=0,
                    min_detection_confidence=0.5, 
                    min_tracking_confidence=0.5) as hands :
        while cap.isOpened():
            ret, frame = cap.read()

            if not ret :
                break

            # 이미지 쓰기 불가 설정 (속도때문에...)
            frame.flags.writeable = False

            # 메시 검출
            results = face_mesh.process(frame)
            frame.flags.writeable = True

            # 랜드마크를 서로 연결 (매시)
            # 랜드마크 점이 검출되었다면
            if results.multi_face_landmarks :
                # 검출 랜드마크를 하나씩 가져온다.
                for face_landmark in results.multi_face_landmarks :
                    # 랜드마크들끼리 선으로 연결
                    mp_drawing.draw_landmarks(
                        image=frame,
                    landmark_list=face_landmark,
                    # 얼굴 전체
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    # 그리기 설정
                    landmark_drawing_spec=drawing_spec,
                    connection_drawing_spec=drawing_spec
                )
                    
            frame.flags.writeable = False
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame)

            # Draw the hand annotations on the image.
            frame.flags.writeable = True
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                    image=frame,
                    landmark_list=hand_landmarks,
                    connections=mp_hands.HAND_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles.get_default_hand_landmarks_style(),
                    connection_drawing_spec=mp_drawing_styles.get_default_hand_connections_style())

            cv2.imshow("face mesh + hand skeleton", cv2.flip(frame, 1))

            key = cv2.waitKey(33)

            if key == 49 :
                break
            
cap.release()
cv2.destroyAllWindows()

## 얼굴, 왼손, 오른손, 동작 검출 (스켈레톤)

In [ ]:
#실습

import cv2
import mediapipe as mp

# 그리기 기능
mp_drawing = mp.solutions.drawing_utils
# 스켈레톤 추출 기능(전체) - 양손, 양손가락, 얼굴, 몸통, 다리, 발
mp_holistic = mp.solutions.holistic

cap = cv2.VideoCapture("./image/face4.mp4")

# 랜드마크 점 찍기
drawing_spec1 = mp_drawing.DrawingSpec(thickness=1, color=(0, 0, 255))
# 선 그리기
drawing_spec2 = mp_drawing.DrawingSpec(thickness=3, color=(255, 0, 0))

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic :
    while cap.isOpened() :
        ret, frame = cap.read()
        
        if not ret :
            break
            
        frame.flags.writeable = False
        results = holistic.process(frame)
        frame.flags.writeable = True
        
        # 몸통/ 다리/ 발 부분의 스켈레톤 그리기
        mp_drawing.draw_landmarks(frame, 
                                  results.pose_landmarks,
                                 mp_holistic.POSE_CONNECTIONS,
                                  landmark_drawing_spec=drawing_spec1,
                                  connection_drawing_spec=drawing_spec2
                                 )
        cv2.imshow("skeleton", frame)
        
        key = cv2.waitKey(33)
        
        if key == 49 :
            break

cap.release()
cv2.destroyAllWindows()

### 손 스켈레톤 검출

In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# For webcam input:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
        if cv2.waitKey(5) & 0xFF == 27:
            break
cap.release()

# 응용 : 자세 분석
- https://github.com/spmallick/learnopencv/tree/master/Posture-analysis-system-using-MediaPipe-Pose

- WorkFlow
  - 각도를 확인할 관심 지점(랜드마크) 검색
  - 표준 샘플 이미지(옆면 이미지)에 대한 분석 수행
  - 좋은 자세와 나쁜 자세에 대한 임계값 범위를 검색
  - 영상에 적용하기
  
<img src="./lecture_image/21_MediaPipe-pose-application.png" width=70%>  

In [ ]:
#실습

- 오프셋 거리 계산
  - 측면보기 상태에서 두 포인트 (눈, 어깨, 골반) 사이의 거리를 계산
  
\begin{align}
distance =  \sqrt{(x2 - x1)^2+(y2 - y1)^2}
\end{align}

In [ ]:
#실습

- 관심 선에서 y축으로 종속된 각도 계산

<img src="./lecture_image/21_calculate_arc.jpg" width=40%>

\begin{align}
\theta = \arccos (\frac{\vec{P_{12}}.\vec{P_{13}}}{|\vec{P_{12}}|.|\vec{P_{13}}|})
= \arccos (\frac{y_1^2 - y_1.y_2}{y_1\sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2}})
\end{align}

In [ ]:
#실습

In [ ]:
#실습

In [ ]:
#실습

# 응용 : Human Action Recognition using Detectron2 and LSTM
  - https://learnopencv.com/human-action-recognition-using-detectron2-and-lstm/

<table border=1 width=100%>
    <tr><td style="border: 1px solid black; width:600px; height:40px; text-align: center;"><font size=4 color=blue><b>[21차시] 정리하기</b></font></td></tr>       
    <tr><td style="border: 1px solid black; text-align: left;"><font size=3>○            
        </font></td></tr>   
</table>